In [ ]:
### Create raster stack of urban parameters and build a random forest model to test correlation with visual texture classification ###
### Predict types of urban texture in Quito ###

library(raster)
library(randomForest)
library(rgdal)
library(psych)

# Prepare parameters, Cuenca
# File that contains the compactness index
comp_c <- readOGR('')

# Create raster template for rasterization
template1_c <- raster(extent(comp_c@bbox), crs = comp_c@proj4string, res = 10)
template2_c <- raster(extent(comp_c@bbox), crs = comp_c@proj4string, res = 100)

# Compute average compactness per 100x100 m raster cells, resample to 10 m for stack
comp_c <- rasterize(comp_c, field = comp_c$comp, template2_c, fun = mean, background = 0)
comp_c <- resample(comp_c, template1_c, method = 'ngb')

# The roughness index
rough_c <- raster('')
rough_c <- resample(rough_c, template1_c, method = 'bilinear')

# Connectivity, average per 100x100 m cell
conn_c <- readOGR('')
# Rasterize number of intersections - use template with large cell size first to save computation time
conn_c_int <- rasterize(conn_c, field = conn_c$no_interse, template2_c, fun = mean, background = 0)
conn_c_int <- resample(conn_c_int, template1_c, method = 'ngb')
# Rasterize road area
conn_c_area <- rasterize(conn_c, field = conn_c$road_area, template2_c, fun = mean, background = 0)
conn_c_area <- resample(conn_c_area, template1_c, method = 'ngb')
rm(conn_c)

# Density, average per 100x100 m cells
dens_c <- readOGR('')
# Rasterize feature count
dens_c_count <- rasterize(dens_c, field = dens_c$no_feat, template2_c, fun = mean, background = 0)
dens_c_count <- resample(dens_c_count, template1_c, method = 'ngb')
# Rasterize field urban area in percent 
dens_c_area <- rasterize(dens_c, field = dens_c$urban_area, template2_c, fun = mean, background = 0)
dens_c_area <- resample(dens_c_area, template1_c, method = 'ngb')
rm(dens_c)

# Pre-processed remote sensing data
planet_c <- stack('')
ndvi_c <- (planet_c[[4]] - planet_c[[3]]) / (planet_c[[4]] + planet_c[[3]])
ndvi_c <- resample(ndvi_c, template1_c, method = 'bilinear')
rm(planet_c)

# MBI
th_c <- raster('')
th_c <- resample(th_c, template1_c, method = 'bilinear')

# Create stack of urban parameters as raster
myind_c <- stack(comp_c, rough_c, conn_c_int, conn_c_area, dens_c_area, dens_c_count, ndvi_c, th_c)
writeRaster(myind_c, '', format = 'GTiff', overwrite = T)
rm(comp_c, conn_c_area, conn_c_int, dens_c_area, dens_c_count, ndvi_c, rough_c, th_c)

# Prepare parameters, Quito
# File used for feature classification with shape metrics 
comp_q <- readOGR('')

template1_q <- raster(extent(comp_q@bbox), crs = comp_q@proj4string, res = 10)
template2_q <- raster(extent(comp_q@bbox), crs = comp_q@proj4string, res = 100)

# Rasterize compactness, average per 100x100 m cell
comp_q <- rasterize(comp_q, field = comp_q$comp, template2_q, fun = mean, background = 0)
comp_q <- resample(comp_q, template1_q, method = 'ngb')

# Roughness, Quito
rough_q <- raster('')
rough_q <- resample(rough_q, template1_q, method = 'bilinear')

# Connectivity, 100x100 m
conn_q <- readOGR('')
# Number of intersections
conn_q_int <- rasterize(conn_q, field = conn_q$no_interse, template2_q, fun = mean, background = 0)
conn_q_int <- resample(conn_q_int, template1_q, method = 'ngb')
# Road area
conn_q_area <- rasterize(conn_q, field = conn_q$road_area, template2_q, fun = mean, background = 0)
conn_q_area <- resample(conn_q_area, template1_q, method = 'ngb')
rm(conn_q)

# Density, 100x100 m
dens_q <- readOGR('')
# Feature count
dens_q_count <- rasterize(dens_q, field = dens_q$no_feat, template2_q, fun = mean, background = 0)
dens_q_count <- resample(dens_q_count, template1_q, method = 'ngb')
# Built area
dens_q_area <- rasterize(dens_q, field = dens_q$urban_area, template2_q, fun = mean, background = 0)
dens_q_area <- resample(dens_q_area, template1_q, method = 'ngb')
rm(dens_q)

# Load, or compute, NDVI
ndvi_q <- raster('')
ndvi_q <- resample(ndvi_q, template1_q, method = 'ngb')

# MBI
th_q <- raster('')
th_q <- resample(th_q, template1_q, method = 'ngb')

# Stack urban parameters
myind_q <- stack(comp_q, rough_q, conn_q_int, conn_q_area, dens_q_area, dens_q_count, ndvi_q, th_q)
writeRaster(myind_q, '', format = 'GTiff', overwrite = T)
rm(comp_q, rough_q, conn_q_int, conn_q_area, dens_q_area, dens_q_count, ndvi_q, th_q, template2_q)

# Prepare and train random forest model
# Load reference texture
# Make sure there is no class with factor '0': All class names will shift
ref_text <- raster('')
# Create sample of 5% of the cells
set.seed(1)
mysample <- sample(1:length(ref_text), length(ref_text)/100 * 5)

# Urban parameter values, Cuenca
valuetable <- as.data.frame(getValues(myind_c))
names(valuetable) <- c('compactness', 'roughness', 'no_intersections', 'road_area', 'density_area', 'density_no', 'NDVI', 'MBI')
valuetable[is.na(valuetable)] <- 0

# Subset parameter values, Cuenca, to the selected 5% for training
mytrain_ind <- valuetable[mysample,]
# Get types of urban texture of traaining pixels
typestable <- as.data.frame(getValues(ref_text))
names(typestable) <- c('type')
mytrain_types <- typestable[mysample,]

train_df <- cbind(mytrain_types, mytrain_ind)
train_df$mytrain_types <- factor(train_df$mytrain_types)

# Create and train random forest model with 500 trees (default)
rf <- randomForest(x = mytrain_ind, y = train_df$mytrain_types, importance = T)
save(rf, file = '')

# Predict types in Cuenca
test <- predict(rf, newdata = valuetable)
# Assign values to the template to get a raster with the predicted values
test_r <- setValues(template1_c, test)
writeRaster(test_r, '', format = 'GTiff', overwrite = T)

# Compare to reference texture for accuracy estimate
pred <- getValues(test_r)
vali <- getValues(ref_text)
# Exclude training pixels
pred_sub <- pred_sub[-mysample]
vali_sub <- vali_sub[-mysample]
validation_table <- as.data.frame(cbind(pred_sub, vali_sub))
# Calculate Cohen's Kappa coefficient
k <- cohen.kappa(validation_table)

# Calculate overall accuracy
correct <- 0
for(i in 1:length(ref_sub)){
  if(ref_sub[i] == pred_sub[i]){
    correct <- correct + 1
  }
}

acc <- correct / length(ref_sub)

# Predict texture in Quito
values_quito <- as.data.frame(getValues(myind_q))
names(values_quito) <- c('compactness', 'roughness', 'no_intersections', 'road_area', 'density_area', 'density_no', 'NDVI', 'MBI')
values_quito[is.na(values_quito)] <- 0
texture_q <- predict(rf, newdata = values_quito)
texture_q_r <- setValues(template1_q, texture_q)
# Apply mask to result
mask_q <- raster('')
mask_q <- resample(mask_q, texture_q_r, method = 'ngb')
texture_q_r <- overlay(texture_q_r, mask_q, fun = function(a, b){ifelse(b == 0, NA, a)})
writeRaster(texture_q_r, '', format = 'GTiff', overwrite = T)